Mount to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install new libraries and import other appropriate libraries

In [3]:
pip install contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 46.4 MB/s eta 0:00:00


In [ ]:
pip install mapclassify

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import contextily as ctx
import rasterio as rio
from rasterio import plot

# Task 1- Read dataset using pandas

In [5]:
Chicago_data_parcels = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/chicago_parcels/Chicago_parcels.shp')

# Task 2- Display the first few rows of the data set and investigate available attributes

Print the first 10 rows

In [7]:
Chicago_data_parcels.head(10)

,POLY_ID,ID,AREA,PIN_OLD,PIN,geometry
0,1,93,0.01,836300010,836300010,"POLYGON ((1091182.694 1942890.287, 1091173.159..."
1,2,94,0.00,836300011,836300011,"POLYGON ((1092430.298 1943191.955, 1092463.452..."
2,3,96,0.02,836300013,836300013,"POLYGON ((1092323.996 1942962.610, 1092301.306..."
3,4,95,0.01,836300012,836300012,"POLYGON ((1092024.119 1942447.762, 1091997.190..."
4,5,92,0.01,836300009,836300009,"POLYGON ((1091970.533 1942338.191, 1091929.082..."
5,6,98,0.00,836301007,836301007,"POLYGON ((1091889.266 1942159.981, 1091833.063..."
6,7,97,0.02,836301003,836301003,"POLYGON ((1091753.543 1941824.459, 1091716.520..."
7,8,5,0.03,836100006,836100006,"POLYGON ((1092172.233 1942219.598, 1092135.727..."
8,9,7,0.00,836101008,836101008,"POLYGON ((1092423.468 1945012.166, 1092400.259..."
9,10,17,0.01,836101028,836101028,"POLYGON ((1092434.770 1945796.435, 1092773.374..."


Investigate data types of the different attributes

In [ ]:
print("The POLY_ID attribute is of type", type("POLY_ID"))
print("The ID attribute is of type", type("ID"))
print("The Area attribute is of type", type("AREA"))
print("The PIN_OLD attribute is of type", type("PIN"))
print("The geometry attribute is of type", type("geometry"))

The POLY_ID attribute is of type <class 'str'>
The ID attribute is of type <class 'str'>
The Area attribute is of type <class 'str'>
The PIN_OLD attribute is of type <class 'str'>
The geometry attribute is of type <class 'str'>


Describe this data (additional general description)

In [14]:
Chicago_data_parcels.info()
Chicago_data_parcels.describe()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 592521 entries, 0 to 592520
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   POLY_ID   592521 non-null  int64   
 1   ID        592521 non-null  int64   
 2   AREA      592521 non-null  float64 
 3   PIN_OLD   592521 non-null  int64   
 4   PIN       592521 non-null  int64   
 5   geometry  592521 non-null  geometry
dtypes: float64(1), geometry(1), int64(4)
memory usage: 27.1 MB


,POLY_ID,ID,AREA,PIN_OLD,PIN
count,592521.000000,592521.000000,592521.000000,5.925210e+05,5.925210e+05
mean,296261.000000,296547.003929,0.000060,1.013798e+09,9.918984e+08
std,171046.223763,171071.776698,0.001704,1.426036e+09,1.448937e+09
min,1.000000,1.000000,0.000000,-2.147484e+09,-2.147484e+09
25%,148131.000000,148426.000000,0.000000,1.306111e+09,1.305426e+09
50%,296261.000000,296556.000000,0.000000,1.433316e+09,1.433303e+09
75%,444391.000000,444690.000000,0.000000,1.914209e+09,1.914209e+09
max,592521.000000,592849.000000,0.370000,2.132213e+09,2.132214e+09


# Task 3- Create a map of the tax parcel polygons in Chicago

In [6]:
pip install lonboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 560.5/560.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 621.3/621.3 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not

In [8]:
from lonboard import Map, SolidPolygonLayer

ValueError: ignored

The code above consistently prints "Value Error". I was unable to continue with the assignment from here wtihout this.